# Demand prediction baseline solution

Victor Kantor, xead.wl@gmail.com

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor

train = pd.read_csv("train.tsv")
test = pd.read_csv("test.tsv")
sample_submission = pd.read_csv("sample_submission.tsv")

In [2]:
def smape(y1, y2):
    return 1.0 * np.mean(2 * abs(y1 - y2)/(abs(y1) + abs(y2)))

In [3]:
all_data = pd.concat([train, test])

ohe = OneHotEncoder(sparse=False)

X = ohe.fit_transform(all_data['item_id'].values.reshape(-1, 1))
X = pd.DataFrame(X, columns=['good = ' + str(x) for x in range(X.shape[1])])
all_data.index = X.index
new_all_data = pd.concat([all_data, X], axis=1)

new_train_init = new_all_data.drop(['item_id', 'Num'], axis=1)[:len(train)]
new_test = new_all_data.drop(['item_id', 'Num'], axis=1)[len(train):]
new_test = new_test.drop(['y'], axis=1)

In [4]:
new_train = new_train_init.take(np.random.permutation(len(new_train_init)))
new_trainX = new_train.drop(['y'], axis=1)
new_trainy = new_train['y']

In [5]:
model = RandomForestRegressor(n_estimators = 17, criterion='mse', min_samples_split = 3)

In [6]:
model.fit(new_trainX, new_trainy)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=3, min_weight_fraction_leaf=0.0,
           n_estimators=17, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

In [7]:
preds = model.predict(new_test)

print len(preds)
print len(sample_submission)
print preds[:10]
sample_submission['y'] = preds
sample_submission.to_csv("baseline_submission_ohe.tsv", sep=',', index=False)

2016
2016
[  2.50530392e+03   2.77155196e+04   3.25545006e+05   3.50794206e+04
   2.70098039e+01   1.44877422e+05   8.88451360e+04   8.37976527e+04
   9.05993431e+04   2.60027941e+03]


In [8]:
preds = model.predict(new_trainX[len(new_trainX)/12*11:])
print smape(preds, new_trainy[len(new_trainy)/12*11:])

0.0997558872748


print smape(preds, new_trainy[len(new_trainy)/12*11:])

на 20 естиматорах - 0.132268540104

на 17  - 0.130200884152